# LAB 4. TOPIC MODELING 

### **<font color=green>INSTRUCTIONS:</font>** <br> 

**<font color=green> 1. Look for EXERCISES in the script (3 in total).</font>** <br>

**<font color=green> 2. Each student INDIVIDUALLY uploads this script with their answers embedded to Canvas by the end of the lab session or by Wednesday, 11:59pm CT (St. Louis time).</font>** 

### **Lab Objectives**

1. Learn how to estimate a topic model in Python (using the sklearn package)
2. Get familiar with the output of a topic model
3. Visualize topics in a text corpus
4. Evaluate and discriminate between topic models

### **Session Prep**
Below we install the modules we need and define the text normalization function we used in Lab 3, as well as two addtional function we need for today only.

In [ ]:
#the module 'sys' allows istalling module from inside Jupyter
import sys

!{sys.executable} -m pip install numpy
import numpy as np

!{sys.executable} -m pip install pandas
import pandas as pd

#Natrual Language ToolKit (NLTK)
!{sys.executable} -m pip install nltk
import nltk

!{sys.executable} -m pip install sklearn
from sklearn import metrics
#from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import  CountVectorizer #bag-of-words vectorizer 
from sklearn.decomposition import LatentDirichletAllocation #package for LDA

# Plotting tools

from pprint import pprint
!{sys.executable} -m pip install pyLDAvis #visualizing LDA
import pyLDAvis
import pyLDAvis.sklearn

import matplotlib.pyplot as plt
%matplotlib inline

#define text normalization function
%run ./Text_Normalization_Function.ipynb #defining text normalization function

#ignore warnings about future changes in functions as they take too much space
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

Below we define two functions that will display the results of fitting a topic model, to be used later:

*Note: these functions are not the focus of the lab, therefore we'll not be discussing them, but you are welcome to explore and dig into them later if you prefer.*

In [2]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        
def get_topic_words(vectorizer, lda_model, n_words):
    keywords = np.array(vectorizer.get_feature_names())
    topic_words = []
    for topic_weights in lda_model.components_:
        top_word_locs = (-topic_weights).argsort()[:n_words]
        topic_words.append(keywords.take(top_word_locs).tolist())
    return topic_words

### **Toy Data Example**
We'll start with working on a toy dataset. It will allow us to grasp the full results of a topic model before moving on to high-dimensional realistic data.

#### **Define and Prep Toy Data**
Let's use the toy corpus on animals and programming similar to one in the lecture. Let's define it:

In [ ]:
toy_corpus = ["The fox jumps over the dog", 
              "The fox is very clever and quick", 
              "The dog is slow and lazy", 
              "The cat is smarter than the fox and the dog but it can never learn Java", 
              "Python is an excellent programming language", 
              "Java and Ruby are other programming languages", 
              "Python and Java are very popular programming languages", 
              "Python programs are smaller than Java programs"]

Let's **normalize** our toy_corpus and call the normalized corpus **normalized_toy_corpus**:

In [ ]:
normalized_toy_corpus = normalize_corpus(toy_corpus) 

Since for topic modeling we need text data in the **Bag-of-Words** representation, let's **vectorize** our normalized_toy_corpus and call it **bow_toy_corpus**:

In [ ]:
#define the bag-of-words vectorizer:
bow_vectorizer = CountVectorizer()

#vectorize the normalized data:
bow_toy_corpus = bow_vectorizer.fit_transform(normalized_toy_corpus)

Have a look at the Bag-of-Words representation of our corpus: **It never hurts to know how you data look like :)** Note absence of stopwords and other differences with the raw data:

In [ ]:
pd.DataFrame(data = bow_toy_corpus.todense(), columns = bow_vectorizer.get_feature_names())

,cat,clever,dog,excellent,fox,java,jump,language,lazy,learn,never,popular,program,programming,python,quick,ruby,slow,small,smarter
0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
3,1,0,1,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1
4,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0
5,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0
6,0,0,0,0,0,1,0,1,0,0,0,1,0,1,1,0,0,0,0,0
7,0,0,0,0,0,1,0,0,0,0,0,0,2,0,1,0,0,0,1,0


#### **Topic Model (via Latent Dirichlet Allocation) on Toy Data**
Now let's **model topics** in our toy data. Given that the toy corpus is so small, we know all "topics" it contains (**what are they?**) and it will be easy for us:<br> 1) to check if the topic model results make sense; <br>2) see all the results that the topic model produces.  <br><br>
We will be using the **LatentDirichletAllocation** function which we already imported earlier (see Session Prep). The function has the following **parameters** to be set:
1. Number of topics to model: **n_components**
2. Parameter vector for the Dirichlet distribution for *topics*: **doc_topic_prior**
3. Parameter vector for the Dirichlet distribution for *words* in a topic: **topic_word_prior**

Notes on **parameter vectors for the Dirichlet distributions**: <br>
1. Although the Dirichlet distribution parameters are represented by a **vector**, for simplicity we provide one number for each parameter vector. For example, if we set the number of topics to 2 (n_components=2), the parameter vector for the Dirichlet distribution for *topics* should be a two-dimensional vector. We set doc_topic_prior=0.5 and the LatentDirichletAllocation function internally creates a two-dimensional vector (0.5,0.5). Similar logic applies to the parameter vector for the Dirichlet distribution for *words* (the dimensionality of that parameter vector is equal to the number of terms in the text corpus, which is typically very large).<br><br>
2. Remember, that we need **sparsity** in the distribution of topics across documents (i.e., some documents have a zero probability of containing some of the topics) and *sparsity* in the distribution of words in topics (i.e., some words have zero probability to be present in some topics). To induce sparsity, we need to set **doc_topic_prior** and **topic_word_prior** between 0 and 1.

Now, let's set the parameters and estimate the topic model:

In [ ]:
lda_toy_corpus = LatentDirichletAllocation(n_components=2, max_iter=500,
                                           doc_topic_prior = 0.9,
                                           topic_word_prior = 0.9).fit(bow_toy_corpus)

Display results by showing 15 **most frequent (top)** words for each topic (we use **function display_topics** defined in Session Prep):

In [ ]:
no_top_words = 15
display_topics(lda_toy_corpus, bow_vectorizer.get_feature_names(), no_top_words)

Topic 0:
java python language programming program popular small excellent ruby dog fox lazy slow clever quick
Topic 1:
fox dog smarter never learn cat jump quick clever lazy slow java programming language python


Note that topics do not have names or labels. **Topics are just collections of words**, following the definition of a topic in text mining. <br><br> To be precise, topics are **word vectors**, where each vector element is the **weight** (relative frequency) of the word in a topic. Let's have a look at those "word vectors". Can you see below that each word vector (topic) is a **simplex**?

In [ ]:
word_weights = lda_toy_corpus.components_ / lda_toy_corpus.components_.sum(axis=1)[:, np.newaxis]
pd.DataFrame(word_weights.T, index = bow_vectorizer.get_feature_names()).T

,cat,clever,dog,excellent,fox,java,jump,language,lazy,learn,never,popular,program,programming,python,quick,ruby,slow,small,smarter
0,0.025956,0.026564,0.026766,0.050982,0.026766,0.109888,0.026407,0.105756,0.026564,0.025956,0.025956,0.051208,0.078480,0.105756,0.105892,0.026564,0.050860,0.026564,0.051163,0.025956
1,0.055089,0.054435,0.113486,0.028201,0.113486,0.053816,0.054604,0.028622,0.054435,0.055089,0.055089,0.027959,0.028293,0.028622,0.028475,0.054435,0.028333,0.054435,0.028007,0.055089


### **<font color=green>EXERCISE 1:</font>**

**<font color=green>1.1. Adjust topic_word_prior (beta) and observe the changes in topic representation. Try these values: 1) topic_word_prior = 10; 2) topic_word_prior = 0.1. What changes in the topic model results do you observe in these two cases? Why? </font>**<br><br> 

<font color=green> *Note: You can also adjust doc_topic_prior, but you will be able to see the restults of this change when we compute shares of topics by document later* </font> <br><br><br>

**<font color=green>1.2. You are likely to be not 100% satisfied with the model performance, even after you set topic_word_prior to the recommended value of below 1. The word "java" might still be appearing among the top words in the "animals" topic. Why? Looking at the text corpus might help to find the answer.</font>**

**Answer 1.1:** <br>

<font color=blue>Code:<font>

In [ ]:
#topic_word_prior = 10

lda_toy_corpus = LatentDirichletAllocation(n_components=2, max_iter=500,
                                           doc_topic_prior = 0.9,
                                           topic_word_prior = 10).fit(bow_toy_corpus)
no_top_words = 15
display_topics(lda_toy_corpus, bow_vectorizer.get_feature_names(), no_top_words)
word_weights = lda_toy_corpus.components_ / lda_toy_corpus.components_.sum(axis=1)[:, np.newaxis]
df1 = pd.DataFrame(word_weights.T, index = bow_vectorizer.get_feature_names()).T
df1

Topic 0:
java dog fox programming language python program excellent slow lazy clever quick smarter learn never
Topic 1:
java python language programming fox dog program small jump ruby popular smarter learn never cat


,cat,clever,dog,excellent,fox,java,jump,language,lazy,learn,never,popular,program,programming,python,quick,ruby,slow,small,smarter
0,0.048390,0.048391,0.052998,0.048392,0.052998,0.055290,0.048383,0.052994,0.048391,0.048390,0.048390,0.048384,0.050681,0.052994,0.052993,0.048391,0.048383,0.048391,0.048382,0.048390
1,0.048384,0.048383,0.052993,0.048382,0.052993,0.055309,0.048391,0.052996,0.048383,0.048384,0.048384,0.048390,0.050701,0.052996,0.052997,0.048383,0.048391,0.048383,0.048392,0.048384


In [ ]:
#topic_word_prior = 0.1

lda_toy_corpus = LatentDirichletAllocation(n_components=2, max_iter=500,
                                           doc_topic_prior = 0.9,
                                           topic_word_prior = 0.1).fit(bow_toy_corpus)
no_top_words = 15
display_topics(lda_toy_corpus, bow_vectorizer.get_feature_names(), no_top_words)
word_weights = lda_toy_corpus.components_ / lda_toy_corpus.components_.sum(axis=1)[:, np.newaxis]
df2 = pd.DataFrame(word_weights.T, index = bow_vectorizer.get_feature_names()).T
df2

Topic 0:
java fox program dog smarter never learn cat quick clever small popular python jump ruby
Topic 1:
programming language python dog excellent lazy slow ruby jump popular java small fox program clever


,cat,clever,dog,excellent,fox,java,jump,language,lazy,learn,never,popular,program,programming,python,quick,ruby,slow,small,smarter
0,0.053701,0.053701,0.077316,0.004882,0.151339,0.200158,0.004884,0.004883,0.004882,0.053701,0.053701,0.053698,0.10252,0.004883,0.004884,0.053701,0.004883,0.004882,0.053701,0.053701
1,0.005709,0.005709,0.092273,0.062798,0.005710,0.005711,0.062796,0.176977,0.062798,0.005709,0.005709,0.005713,0.00571,0.176977,0.176975,0.005709,0.062798,0.062798,0.005710,0.005709


<font color=blue>Discussion: <font>

We can find that in the first data frame with topic_word_prior = 10, every word has almost the same weight in each topic, which makes them not discriminate for its topic. 

While in the second data frame, where topic_word_prior = 0.1, the result is quite differentiating, and we can easily know which word occurs more frequently than others in that topic.

**Answer 1.2:**

<font color=blue>Discussion:<font>

Because 'Java' appeared in the same document with other animal names.

### **Topic Modeling on Real Data**

The dataset here is the one we used for doing Text Classification in Lab 3. The newspaper blog posts have 4 topics: **atheism, religion, computer graphics, and space science**. Of course, we will *not use* class label information for topic modeling.

Download the data and set up the data (**news_corpus**):

In [6]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
dataset = fetch_20newsgroups(shuffle=True, 
                             random_state=1, 
                             categories = categories, 
                             remove=('headers', 'footers', 'quotes'))
news_corpus = dataset.data

Now, let's normalize the corpus and create the Bag-of-Words representation of the data. We'll limit the number of features to **1000 most frequent features** to compute the topic model faster. 

In [7]:
#normalize data
normalized_corpus_news = normalize_corpus(news_corpus)

#define a Bag-of-Words vecgtorizer
bow_vectorizer_news = CountVectorizer(max_features=1000)

#vectorize data
bow_news_corpus = bow_vectorizer_news.fit_transform(normalized_corpus_news)

Now let's fit the topic model. We need to **set the number of topics** first. We are *lucky to know* that there are **4 topics** (atheism, religion, computer graphics, and space science) and it will allow us to judge the performance of the topic model better.

**Note**: It will take a couple of minutes for the estimation to finish. The larger the number of iterations (max_iter) you allow for, the longer it takes. You can also find that you might get a slightly different result (for the estimated weights) if you rerun the model (qualitatively, the result is likely to stay the same). This behavior is due to the stochastic nature of the model. Estimation starts from a random guess for the weights and iteratively improves. When we do not allow for a large number of iterations, we are more likely to be affected by the random starting values. 

In [8]:
lda_news = LatentDirichletAllocation(n_components=4, max_iter=100,
                                     doc_topic_prior = 0.25,
                                     topic_word_prior = 0.25).fit(bow_news_corpus)

Display results with top 10 words for each topic:

In [9]:
no_top_words_news = 10
display_topics(lda_news, bow_vectorizer_news.get_feature_names(), no_top_words_news)

Topic 0:
god people think know jesus believe christian good even thing
Topic 1:
launch space year use orbit time satellite first earth like
Topic 2:
space nasa data edu information include research system center list
Topic 3:
image file use program graphic format software jpeg color edu


Display **word vectors** (words are in alphabetical order) for each topic. Each column is a topic:

In [ ]:
word_weights = lda_news.components_ / lda_news.components_.sum(axis=1)[:, np.newaxis]
word_weights_df = pd.DataFrame(word_weights.T, 
                               index = bow_vectorizer_news.get_feature_names(), 
                               columns = ["Topic_" + str(i) for i in range(4)])
word_weights_df.head(10)

,Topic_0,Topic_1,Topic_2,Topic_3
3d,0.000009,0.000012,0.000007,0.004705
able,0.000902,0.000472,0.001240,0.001370
ac,0.000354,0.000058,0.000007,0.001156
accept,0.000009,0.000687,0.002953,0.000293
acceptable,0.000279,0.000012,0.000753,0.000186
access,0.000874,0.000013,0.000197,0.001785
accord,0.000407,0.001315,0.001097,0.000008
account,0.000106,0.001094,0.000478,0.000052
act,0.000797,0.002663,0.001129,0.000009
action,0.000279,0.000013,0.002388,0.000010


Now, **sort by word weights in Topic 0** (descending order) and see the weights by 10 most frequent words in Topic 0:

In [ ]:
word_weights_df.sort_values(by='Topic_0',ascending=False).head(10)

,Topic_0,Topic_1,Topic_2,Topic_3
space,0.036852,0.000012,0.000007,0.000793
nasa,0.014915,0.000012,0.000007,0.000009
launch,0.014410,0.000012,0.000007,0.000008
year,0.010786,0.004084,0.001250,0.000306
satellite,0.010259,0.000012,0.000007,0.000008
orbit,0.009140,0.000012,0.000007,0.000008
system,0.008767,0.000012,0.004525,0.007285
earth,0.008722,0.001132,0.000393,0.000008
use,0.008584,0.004300,0.007439,0.015787
mission,0.008584,0.000217,0.000007,0.000008


### **Topic Model Visualization**

You can **visualize** the topics: topic size, frequency of words in a topic and so on.

In this visualization, you can rank words in a topic by **relevancy**: do you want rare and exclusive terms (i.e. found mostly in that topic) or terms that are used frequently in that topic, but not always exclusive to that topic? Relevancy parameter is λ (0 ≤ λ ≤ 1). You can adjust it:

* small λ highlights potentially rare, but exclusive terms for the selected topic;
* large values of λ (near 1) highlight frequent, but not necessarily exclusive, terms for the selected topic;

Relevancy is measured as: 

    Relevancy = λ log[p(term | topic)] + (1 - λ) log[p(term | topic)/p(term)], 
   
   where p(term | topic) stands for word weight in a topic and p(term) stands for word's weight in a corpus.

Additional information on how to use this visualization:
* http://www.kennyshirley.com/LDAvis/
* https://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf

We installed all the **pyLDAvis** module required for this visualization in Session Prep. Now let's use it:

In [10]:
#prepare to display result in the Jupyter notebook
pyLDAvis.enable_notebook()

#run the visualization [mds is a function to use for visualizing the "distance" between topics]
pyLDAvis.sklearn.prepare(lda_news, bow_news_corpus, bow_vectorizer_news, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
0      269.562408 -124.803398       1        1  43.689168
3        1.782994 -295.079681       2        1  21.875269
1      220.811432 -343.832123       3        1  20.433263
2       50.533993  -76.050934       4        1  14.002300, topic_info=        Term         Freq        Total Category  logprob  loglift
846    space  1045.000000  1045.000000  Default  30.0000  30.0000
416    image   844.000000   844.000000  Default  29.0000  29.0000
329     file   545.000000   545.000000  Default  28.0000  28.0000
576     nasa   414.000000   414.000000  Default  27.0000  27.0000
368      god   782.000000   782.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
61   archive    88.021297   150.624438   Topic4  -5.2236   1.4287
700  program   117.128538   532.450966   Topic4  -4.9379   0.4517
555  mission    95.040338   242.227144   Topic4  -5.1469   1.0304
158      com    88.723524   176.292205   Topic4  -5.2157   1.2793
515     mail    89.908426   242.713142   Topic4  -5.2024   0.9729

[277 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
0         2  0.996187     3d
22        1  0.972901  agree
22        3  0.026118  agree
23        1  0.031462    air
23        3  0.928135    air
...     ...       ...    ...
989       4  0.074280   work
995       2  0.988015     xv
996       1  0.179217   year
996       3  0.744092   year
996       4  0.074863   year

[471 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 2, 3])

### **<font color=green>EXERCISE 2:</font>**
    
**<font color=green>2.1. Fit a topic model with 3 topics (n_components = 3). The script is provided below. Important! Note that the model with three topics is called lda_news_3_topics.</font>**

**<font color=green>2.2. Use the visualization tool to answer the following question: </font>**

> **<font color=green>a) Which topic is the most common / largest topic in the corpus? Can you give a name to that topic? </font>**



> **<font color=green>b) What are 5 most frequent terms for that topic? What are 5 most exclusive terms for that topic? Now, list exclusive but relatively frequent terms by setting lambda = 0.2.</font>**







**<font color=green>2.3. You fit the model with 3 topics, but you know that the dataset has four classes (topics). Which 2 of the 4 classes ('atheism', 'religion', 'computer graphics', 'space science') were grouped together into one topic by the topic model when you fit it with 3 topics? Why?</font>**

Your answer (need to add lines of code related to visualization):

**Answer 2.1**:

Code:

In [11]:
#fit the LDA model with 3 topics
lda_news_3_topics = LatentDirichletAllocation(n_components=3, max_iter=100,
                                     doc_topic_prior = 0.25,
                                     topic_word_prior = 0.25).fit(bow_news_corpus)

<font color=blue>Discussion (if needed):<font>

**Answer 2.2:**

<font color=blue>Code:<font>

In [12]:
#prepare to display result in the Jupyter notebook
pyLDAvis.enable_notebook()

#run the visualization [mds is a function to use for visualizing the "distance" between topics]
pyLDAvis.sklearn.prepare(lda_news_3_topics, bow_news_corpus, bow_vectorizer_news, mds='tsne')

PreparedData(topic_coordinates=                 x            y  topics  cluster       Freq
topic                                                      
2     -1015.700562  1338.981201       1        1  48.478796
0      -387.996002  -373.316162       2        1  27.048214
1     -2184.745605   -60.565800       3        1  24.472990, topic_info=        Term         Freq        Total Category  logprob  loglift
846    space  1044.000000  1044.000000  Default  30.0000  30.0000
416    image   847.000000   847.000000  Default  29.0000  29.0000
329     file   547.000000   547.000000  Default  28.0000  28.0000
479   launch   399.000000   399.000000  Default  27.0000  27.0000
368      god   784.000000   784.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
913     time   185.787260   654.529235   Topic3  -5.0350   0.1483
700  program   171.660997   533.342526   Topic3  -5.1140   0.2740
399     high   136.699803   246.727363   Topic3  -5.3418   0.8171
787  science   134.203468   237.643382   Topic3  -5.3602   0.8362
421  include   131.774595   381.787582   Topic3  -5.3785   0.3438

[212 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0         2  0.993940         3d
22        1  0.978094      agree
22        3  0.019562      agree
24        1  0.011871  algorithm
24        2  0.985286  algorithm
...     ...       ...        ...
993       2  0.473654      write
993       3  0.060651      write
996       1  0.309933       year
996       2  0.022623       year
996       3  0.667375       year

[316 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])

<font color=blue>Discussion:<font>

a) Topic 1 is the most common / largest topic in the corpus. Let's name it religion.


b) 'God', 'people', 'think', 'know', 'like' are 5 most frequent terms for that topic.
And 'God', 'people', 'think','jesus', 'christian' are 5 most exclusive terms for that topic.

**Answer 2.3:**

In [ ]:
no_top_words_news = 10 
display_topics(lda_news_3_topics, bow_vectorizer_news.get_feature_names(), no_top_words_news)

Topic 0:
space nasa launch year satellite orbit system use earth mission
Topic 1:
god people think know like jesus thing good believe even
Topic 2:
image file use edu program software graphic format data jpeg


In [ ]:
no_top_words_news = 10 
display_topics(lda_news, bow_vectorizer_news.get_feature_names(), no_top_words_news)

Topic 0:
space nasa launch year satellite orbit system earth use mission
Topic 1:
jesus god christian people know bible good day child time
Topic 2:
think people god like know well believe could thing use
Topic 3:
image file use edu program software graphic format data jpeg


We can see from the above, that 'atheism' and 'religion' were grouped together into one topic. Because these two topics share some same words.

<font color=blue>Discussion:<font>


### **How To Find Dominant Topic in a Document**

Each document contains several topics. One of the topics is **dominant**, i.e. it is the largest topic in the document. That topic gives you an answer to the question: **What is this document about?** In other words, the document's dominant topic **summarizes** the document. 

Let's assign a dominant topic to **each document** in our corpus. Weights in a word vector for a topic provide a measure of association for the word with the topic. If you sum weights for a particular topic across all words in a document, you'll get the weight of that topic in the document.

The attribute **.transform** to our function **lda_news** computes the weights of each topic in documents: 

In [ ]:
lda_news_topic_weights = lda_news.transform(bow_news_corpus)

Let's convert lda_news_topic_weights into a nice-looking dataframe and have a look at the computed topic weights in documents:

In [ ]:
#array of document "names" and topic "names" ("names" are just indecies)
doc_names = ["Doc_" + str(i) for i in range(len(normalized_corpus_news))]
topic_names = ["Topic_" + str(i) for i in range(4)]

#convert to dataframe
df_document_topic = pd.DataFrame(np.round(lda_news_topic_weights, 4), columns=topic_names, index=doc_names)
df_document_topic.head(5)

,Topic_0,Topic_1,Topic_2,Topic_3
Doc_0,0.0004,0.9987,0.0005,0.0004
Doc_1,0.0027,0.0026,0.2492,0.7455
Doc_2,0.0107,0.0111,0.9675,0.0106
Doc_3,0.2366,0.0096,0.0095,0.7444
Doc_4,0.0500,0.0501,0.0501,0.8497


The topic with the highest weight in each document is a **dominant topic**. The weights across the 4 topics sum up to 1. Let's add a column that shows dominant topic for each document:

In [ ]:
#vector of indecies for columns with the highest value by each row in df_document_topic
dominant_topic = np.argmax(df_document_topic.values, axis=1)

#add dominant_topic as a column to df_document_topic
df_document_topic['dominant_topic'] = dominant_topic
df_document_topic.head(5)

,Topic_0,Topic_1,Topic_2,Topic_3,dominant_topic
Doc_0,0.0004,0.9987,0.0005,0.0004,1
Doc_1,0.0027,0.0026,0.2492,0.7455,3
Doc_2,0.0107,0.0111,0.9675,0.0106,2
Doc_3,0.2366,0.0096,0.0095,0.7444,3
Doc_4,0.0500,0.0501,0.0501,0.8497,3


### **Topic Model Evaluation: Log-likelihood, Perplexity and Coherence Scores**

Log-likelihood, Perplexity and Coherence Score are **measures of performance** for a topic model. They are used for comparing and discriminating between topic models estimated on the same data. Log-likelihood, perplexity and coherence scores **do not have** a baseline or a threshold values and therefore are useful only for comparing models. 

How do you specify different models? You can set **different number of topics** and also play with the **parameters of the Dirichlet distributions**. 

#### **Log-Likelihood Score**

To compute the log-likelihood score we use the **.score** attribute of our defined and fitted LDA function:

In [ ]:
print("Log-Likelihood (higher values are better): ", lda_news.score(bow_news_corpus))

Log-Likelihood (higher values are better):  -740898.1461126149


#### **Perplexity Score**

To compute the Perplexity score we use the **.perplexity** attribute of our defined and fitted LDA function:

In [ ]:
print("Perplexity (lower values are better): ", lda_news.perplexity(bow_news_corpus))

Perplexity (lower values are better):  574.0806732632797


####Coherence Score

We will use a function CoherenceModel() from the gensim module (you can also explore that package as it can be used to estimate an LDA model). The sklearn module does not have the functionality to compute the coherence score. Let's install the gensim package and the functions needed:

In [ ]:
!{sys.executable} -m pip install gensim
import gensim

from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

The function CoherenceModel() needs as **inputs**:

**1. Dictionary of the corpus**<br>
**2. Corpus with each document represented as Bag-of-Words**<br>
**3. An array of top words for each topic: we'll have top 20 words for each topic** 
  
We will now create those objects:

In [ ]:
#tokenizing the corpus
news_corpus_tokenized = [tokenize_text(normalized_corpus_news[doc_id]) for doc_id in range(len(normalized_corpus_news))]

#Dictionary of the corpus:
news_dictionary = Dictionary(news_corpus_tokenized)

#Bag-of-words representation for each document of the corpus:
news_corpus_bow = [news_dictionary.doc2bow(doc) for doc in news_corpus_tokenized]

#top 20 words for each topic (using the function defined in session prep)
topic_topwords = get_topic_words(vectorizer = bow_vectorizer_news, lda_model = lda_news, n_words=20)

Now let's compute **the coherence score for the model overall**. We use one of the coherence metrics "u-mass" which measures semantic similarity of words in a topic, but there are other metrics as well.

*Note: You can check out different coherence metrics here if you are interested: https://dl.acm.org/doi/abs/10.1145/2684822.2685324*

In [ ]:
cm = CoherenceModel(topics=topic_topwords, 
                    corpus = news_corpus_bow , 
                    dictionary = news_dictionary, coherence='u_mass')
print("Coherence score for the model: ", np.round(cm.get_coherence(), 4))  # get coherence value

Coherence score for the model:  -1.4784


You can also see **coherence scores by topic**:

In [ ]:
print("Coherence score by topic (higher values are better): ", np.round(cm.get_coherence_per_topic(),4))

Coherence score by topic (higher values are better):  [-1.4047 -1.7925 -1.3605 -1.3558]


### **<font color=green>EXERCISE 3</font>**

**<font color=green>Compare the coherence score, perplexity score and the log-likelihood for models with 2, 3, and 4 topics with your human-judgment-based evaluation of those models. What do you find? </font>**<br><br>

**<font color=green>What you need to do:</font>**

**<font color=green>3.1. For model with 4 topics - All code work is done: The model and evaluation metrics are already computed above. You just need to look up the values for the coherence, perplexity and log-likelihood for the model with 4 topics above and discuss what you observe. You might be interested in looking at coherence score by topic as well;</font>**

**<font color=green>3.2. For model with 3 topics - The model is computed in Exercise 2. You need to compute the perplexity, log-likelihood and coherence scores for the model with 3 topics (the lines for the coherence score are provided below) and dicuss your results;</font>**

**<font color=green>3.3. For model with 2 topics - You need to fit the model with 2 topics and compute all 3 evaluation metrics; dicuss your results.</font>**

**Answer 3.1:**

<font color=blue>Discussion:<font>

* Coherence score for the model:  -1.4784
* Coherence score by topic (higher values are better):  [-1.4047 -1.7925 -1.3605 -1.3558]
* Log-Likelihood (higher values are better):  -740898.1461126149
* Perplexity (lower values are better):  574.0806732632797


**Answer 3.2:**

<font color=blue>Code (complete the lines):<font>

In [ ]:
topic_topwords = get_topic_words(vectorizer = bow_vectorizer_news, lda_model = lda_news_3_topics, n_words=20)

cm = CoherenceModel(topics=topic_topwords, 
                    corpus = news_corpus_bow , 
                    dictionary = news_dictionary, coherence='u_mass')
print("Coherence score for the model: ", np.round(cm.get_coherence(), 4))  # get coherence value

print("Log-Likelihood (higher values are better): ", lda_news_3_topics.score(bow_news_corpus))

print("Perplexity (lower values are better): ", lda_news_3_topics.perplexity(bow_news_corpus))

Coherence score for the model:  -1.55
Log-Likelihood (higher values are better):  -750014.4910939496
Perplexity (lower values are better):  620.7554964888775


**Answer 3.3:**

Code:

In [ ]:
  lda_news_2_topics = LatentDirichletAllocation(n_components=2, max_iter=100,
                                     doc_topic_prior = 0.25,
                                     topic_word_prior = 0.25).fit(bow_news_corpus)

In [ ]:
topic_topwords = get_topic_words(vectorizer = bow_vectorizer_news, lda_model = lda_news_2_topics, n_words=20)

cm = CoherenceModel(topics=topic_topwords, 
                    corpus = news_corpus_bow , 
                    dictionary = news_dictionary, coherence='u_mass')
print("Coherence score for the model: ", np.round(cm.get_coherence(), 4))  # get coherence value

print("Log-Likelihood (higher values are better): ", lda_news_2_topics.score(bow_news_corpus))

print("Perplexity (lower values are better): ", lda_news_2_topics.perplexity(bow_news_corpus))

Coherence score for the model:  -1.4145
Log-Likelihood (higher values are better):  -751628.340406743
Perplexity (lower values are better):  629.4051056440891


In [ ]:
no_top_words_news = 10 
display_topics(lda_news_2_topics, bow_vectorizer_news.get_feature_names(), no_top_words_news)

Topic 0:
space image use file program system data edu nasa launch
Topic 1:
god people think know like jesus thing good believe even


<font color=blue>Discussion:<font>



* By looking at the coherence score, the topic 2 model has the highest result.

* While we look at the log-likelihood and perplexity, topic model 4 has the best result.

* Human-judgment: The topic 2 model is not good because it forces the 'space science' and 'computer science' groups to combine while in different fields. Model 4 is great, but I think 'atheism' is a subset of a religious topic. So I like the topic 3 model.

<br>**NOTE:** Generally, you can write a simple script that selects the best topic model **automatically** based on a criterion for "best model" (log-likelihood, perplexity, or coherence score). The script can vary both parameters of the Dirichlet distributions and the number of topics, or just the number of topics.